In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.preprocessing import StandardScaler, OneHotEncoder, \
FunctionTransformer, normalize, PolynomialFeatures, LabelEncoder

from sklearn.linear_model import LinearRegression, LogisticRegression, \
Ridge, Lasso, ElasticNet, LassoCV, RidgeCV, ElasticNetCV

from sklearn.model_selection import train_test_split, cross_validate,\
KFold, cross_val_score, ShuffleSplit, RandomizedSearchCV, GridSearchCV

from sklearn.metrics import mean_squared_error, make_scorer, log_loss, \
confusion_matrix, plot_confusion_matrix, precision_score, recall_score, \
accuracy_score, f1_score, roc_curve, roc_auc_score, classification_report, auc, plot_roc_curve

from sklearn.dummy import DummyRegressor, DummyClassifier

from sklearn.utils import resample

from sklearn.impute import MissingIndicator, SimpleImputer

from sklearn.feature_selection import SelectFromModel

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree

from sklearn import tree

from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

from scipy import stats
from sklearn.naive_bayes import MultinomialNB, GaussianNB

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer,  make_column_selector as selector

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, \
ExtraTreesClassifier, VotingClassifier, StackingRegressor, StackingClassifier

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

import xgboost

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

%matplotlib inline


# Import Data

In [2]:
X_data_df = pd.read_csv('data/x_data.csv')
y_data_df = pd.read_csv('data/target_data.csv')

## Label Encode Y-vars

In [3]:
y_encoded = pd.DataFrame(LabelEncoder().fit_transform(y_data_df['status_group']))

In [4]:
y_encoded.value_counts(normalize=True)

0    0.543081
2    0.384242
1    0.072677
dtype: float64

- 0 = functional
- 1 = functional needs repair
- 2 = non functional

## Concat X & Y so that when I drop Null rows, they match still

In [5]:
df = pd.concat([X_data_df, y_encoded], axis=1)

### Giving the new target col a better name 

In [7]:
df.rename(mapper={0:'target'}, axis=1, inplace=True)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

## Dropping Columns that are not relevant to the search

In [10]:
cols_to_drop = ['scheme_name', 'id', 'date_recorded', 'region', 'funder', 'recorded_by']
df_dropped = df.drop(cols_to_drop, axis=1)

## Cleaning nulls

In [11]:
df_no_nulls = df_dropped.dropna(subset=['subvillage'])

In [16]:
df_no_nulls.isna().sum()

amount_tsh                  0
gps_height                  0
installer                3655
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                  0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting           3333
scheme_management        3869
permit                   3056
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity                    0
quantity_group              0
source                      0
source_type                 0
source_class                0
waterpoint_type             0
waterpoint

## Imputing Missing Vals for remaining columns based on frequency

### Need to ensure that we mark if any cols were imputed

In [13]:
imp = SimpleImputer(strategy='most_frequent', add_indicator=True)

In [14]:
df_clean_impute = imp.fit_transform(df_no_nulls)

### We now have imputed array but no dataframe, need to grab the columns and set up a df (for my own ease, not truly necessary)

In [15]:
cols_to_keep = list(df_no_nulls.columns)

In [18]:
indicator_list = ['installer_indicator', 'public_meeting_indicator', 'scheme_management_indicator', 'permit_indicator']

for x in indicator_list:
    cols_to_keep.append(x)

In [19]:
df_clean_impute = pd.DataFrame(df_clean_impute, columns=cols_to_keep)

# Data Cleaning is complete:

**Things to Note:**
- missing values are imputed or dropped depending on the column
- no data transformation has occured yet. The X and Y labels are still seperate (no leakage)

Need to Check that each column is the correct category

In [39]:
df_clean_impute.columns

Index(['amount_tsh', 'gps_height', 'installer', 'longitude', 'latitude',
       'wpt_name', 'num_private', 'basin', 'subvillage', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group', 'target',
       'installer_indicator', 'public_meeting_indicator',
       'scheme_management_indicator', 'permit_indicator'],
      dtype='object')

In [40]:
df_clean_impute.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year,target
count,59029.000000,59029.000000,59029.000000,5.902900e+04,59029.000000,59029.000000,59029.000000,59029.000000
mean,319.646832,672.299582,34.062263,-5.704219e+00,0.477121,180.941097,1308.488048,0.840841
std,3006.873793,693.281914,6.585080,2.954644e+00,12.274565,472.709908,949.095754,0.949509
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.079614,-8.563705e+00,0.000000,0.000000,0.000000,0.000000
50%,0.000000,377.000000,34.889047,-5.000106e+00,0.000000,30.000000,1986.000000,0.000000
75%,25.000000,1322.000000,37.189623,-3.323780e+00,0.000000,220.000000,2004.000000,2.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,30500.000000,2013.000000,2.000000


In [41]:
num_cols = ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'population', 'construction_year', 'target']

In [42]:
for col in num_cols:
    df_clean_impute[col] = df_clean_impute[col].astype(float)

In [43]:
df_clean_impute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59029 entries, 0 to 59028
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   amount_tsh                   59029 non-null  float64
 1   gps_height                   59029 non-null  float64
 2   installer                    59029 non-null  object 
 3   longitude                    59029 non-null  float64
 4   latitude                     59029 non-null  float64
 5   wpt_name                     59029 non-null  object 
 6   num_private                  59029 non-null  float64
 7   basin                        59029 non-null  object 
 8   subvillage                   59029 non-null  object 
 9   region_code                  59029 non-null  object 
 10  district_code                59029 non-null  object 
 11  lga                          59029 non-null  object 
 12  ward                         59029 non-null  object 
 13  population      

# Time get started with model prep

- Need to set up X and Y vars
- then move to train/test split
- set up a pipeline seperate cat and num cols
- set up pipeline to scale num data
- set up a pipeline to ohe cats

In [44]:
y = df_clean_impute['target']
X = df_clean_impute.drop(['target'], axis =1)

In [45]:
df_clean_impute.columns

Index(['amount_tsh', 'gps_height', 'installer', 'longitude', 'latitude',
       'wpt_name', 'num_private', 'basin', 'subvillage', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group', 'target',
       'installer_indicator', 'public_meeting_indicator',
       'scheme_management_indicator', 'permit_indicator'],
      dtype='object')

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Setting up prelim Pipelines

In [47]:
num_pipe = Pipeline([
    ('ss', StandardScaler())
])

In [48]:
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [49]:
ColTrans = ColumnTransformer(transformers=[
    ('num_pipe', num_pipe, selector(dtype_include=np.number)),
    ('cat_pipe', cat_pipe, selector(dtype_include=object))
], remainder='passthrough')